In [ ]:
import pandas as pd
import numpy as np
import requests
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline
#%matplotlib nbagg
#%matplotlib notebook

sns.set()

# Input 
### PostgreSQL
Henter bilernes registreringsnummer fra flådestyrings tabel

In [ ]:
user = 'xx'
pw = 'xx'
port = 5432
host = 'xx.xx.com'
db = 'xx'

In [ ]:
con = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

In [ ]:
query = """
    SELECT * 
    FROM loriot.indeklima_dekodet
    WHERE eui = 'A81758FFFE030D78'
    """

In [ ]:
df = pd.read_sql_query(query,con=con)

In [ ]:
df.describe()

### Moving avarage and pct change

In [ ]:
periods = 3
window = 3

In [ ]:
df['co2_change'] = df.co2.pct_change(periods=periods)
df['temp_change'] = df.temp.pct_change(periods=periods)
df['humidity_change'] = df.humidity.pct_change(periods=periods)
df['co2_mva'] = df.co2.rolling(window=20).mean()
df['temp_mva'] = df.temp.rolling(window=window).mean()
df['humidity_mva'] = df.humidity.rolling(window=window).mean()
df['co2_mva_change'] = df.co2_mva.pct_change(periods=periods)
df['motion_bool'] = df.motion.apply(lambda x: True if x > 0 else False)

### MOTION AND LIMITED DATARANGE

In [ ]:
motion = df[
    (df.ts > '2018-07-13 09:00:00') & 
    (df.ts < '2018-07-23 16:00:00') 
]

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(511)
plt.plot(motion.ts, motion.motion, 'o', color='g')
plt.title('motion')
plt.subplot(512)
plt.plot(motion.ts, motion.co2, 'o')
plt.title('co2')
plt.ylabel('ppm')
plt.subplot(513)
plt.plot(motion.ts, motion.temp, '-', color='y')
plt.title('temp')
plt.ylabel('grader')
plt.subplot(514)
plt.plot(motion.ts, motion.co2_mva, 'o', color='c')
plt.title('CO2 moving avarage')
plt.ylabel('ppm')
plt.subplot(515)
plt.plot(motion[motion.motion_bool == False].ts, motion[motion.motion_bool == False].co2_mva_change, 'o', color='c')
plt.plot(motion[motion.motion_bool == True].ts, motion[motion.motion_bool == True].co2_mva_change, 'o', color='m')
plt.title('CO2 moving avarage pct change')
plt.ylabel('pct')

### DATERANGE

In [ ]:
pd.date_range(df.ts.min(), df.ts.max(), freq='10min', normalize=True)

### RESAMPLING
http://benalexkeen.com/resampling-time-series-data-with-pandas/

In [ ]:
motion.set_index('ts', inplace=True)

Create empty DF for resampling

In [ ]:
resample = pd.DataFrame()

Set interval

In [ ]:
rule = '5Min'

Resample cols and add time and date for pivot

In [ ]:
resample['eui'] = motion['eui'].resample(rule).first()
resample['temp'] = motion['temp'].resample(rule).mean()
resample['humidity'] = motion['humidity'].resample(rule).mean()
resample['motion'] = motion['motion'].resample(rule).sum()
resample['co2'] = motion['co2'].resample(rule).mean()
resample['motion_bool'] = resample.motion.apply(lambda x: 1 if x > 0 else 0)
resample['time'] = resample.index.time
resample['date'] = resample.index.date



https://codereview.stackexchange.com/a/155133

In [ ]:
resample['motion_bool'].diff()

In [ ]:
# find when the state changes
run_change = resample['motion_interval'] = resample['motion_bool'].diff().fillna(0)

# get the step lengths
step_length = resample.reset_index().ts.diff()

# loop and get the change since last state change
since_change = []
current_delta = 0
for is_change, delta in zip(run_change, step_length):
    current_delta = 0 if is_change != 0 else \
        current_delta + delta.total_seconds() / 60.0
    since_change.append(current_delta)

# add this data to the data frame
resample['time_since_change'] = pd.Series(since_change).fillna(0).values

Only show business hours mon-fri, 9-16

In [ ]:
bday = resample[(~resample.index.dayofweek.isin([5,6])) ].between_time('9:00', '16:00')

In [ ]:
bday

In [ ]:
pivot = bday.pivot(
    index = 'time', 
    columns = 'date', 
    values = 'motion_bool'
)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(pivot, cbar=False, cmap='Reds')

In [ ]:
motion[motion.index >= '2018-07-19 09:45:00'][['eui', 'temp', 'humidity', 'motion', 'co2']].head(20)

# Viz

In [ ]:
sns.pairplot(motion[['temp', 'co2', 'humidity', 'motion_bool', 'co2_mva_change']], diag_kind='kde', hue='motion_bool')

In [ ]:
d = datetime.now() 
d = d.replace(month=d.month - 1 if d.month > 1 else 12)

In [ ]:
lm = df[df['ts'] > '2018-07-12']

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

import cufflinks as cf

init_notebook_mode(connected=True)

In [ ]:
iplot(motion)

In [ ]:
from math import cos
from IPython.display import SVG, HTML
xy_chart = pygal.XY()
xy_chart.title = 'XY Cosinus'
xy_chart.add('x = cos(y)', [(cos(x / 10.), x / 10.) for x in range(-50, 50, 5)])
xy_chart.add('y = cos(x)', [(x / 10., cos(x / 10.)) for x in range(-50, 50, 5)])
xy_chart.add('x = 1',  [(1, -5), (1, 5)])
xy_chart.add('x = -1', [(-1, -5), (-1, 5)])
xy_chart.add('y = 1',  [(-5, 1), (5, 1)])
xy_chart.add('y = -1', [(-5, -1), (5, -1)])
HTML(html_pygal.format(pygal_render=line_chart.render()))


In [ ]:
plt.figure(figsize=(10,8))
plt.plot(last_month.ts, last_month.temp)

In [ ]:
lm

In [ ]:
change.co2.describe()

In [ ]:
import sys

In [ ]:
sys.path.append('/Users/andersbarfod/Documents/git/')

In [ ]:
import connections as con

In [ ]:
con.sql_to_dataframe('DISTRIBUTION', 'select * from proj_belysning.belysning limit 10')